In [37]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)

In [38]:
obwody = pd.read_csv('./obwody_glosowania_utf8.csv', sep=';')

In [39]:
obwody = obwody[['Gmina','TERYT gminy','Numer', 'Kod pocztowy']]

In [40]:
obwody = obwody.dropna()

In [41]:
obwody.head(2)

,Gmina,TERYT gminy,Numer,Kod pocztowy
0,m. Bolesławiec,20101.0,1,59-700
1,m. Bolesławiec,20101.0,2,59-700


# Klastrowanie

Z artykułu:
1. Początkowe grupowanie oparto na pierwszych dwóch cyfrach kodu pocztowego (np. „30” dla obszaru Krakowa).
2. Jeżeli powstała grupa zawierała od 10 do 16 komisji, została zaakceptowana bez zmian.
3. Grupy liczące mniej niż 10 komisji odłożono do późniejszego łączenia.
4. Grupy przekraczające 16 komisji dzielono rekurencyjnie, dodając kolejne cyfry kodu
    pocztowego (np. z „30” → „301” → „3011” i dalej, aż do pełnych pięciu cyfr).
5. Pozostałe małe grupy łączono z najbliższymi sąsiadami mającymi ten sam prefiks, przy czym
    priorytetem była ciągłość przestrzenna i zrównoważona liczebność grup.
    W odróżnieniu od wcześniejszego podejścia, które dopuszczało grupy o wielkości 10–25 komisji,
    niniejsze badanie przyjęło węższy zakres docelowy: od 10 do 16 komisji na grupę
   
   
Komentarz: Algorytm z artykułu nie jest deterministyczny (niejednoznacznie wskazuje co się dzieje z grupami liczącymi mniej niż 10 komisji do ponownego połączenia) nie da się go więc odtworzyć na podstawie jego opisu. Dlatego właśnie rzetelne artykułu zawierają kod źródłowy umożliwiający reprodukcję.

In [42]:
import pandas as pd
from collections import defaultdict

def assign_groups(df: pd.DataFrame,
                  code_col: str = "kod",
                  min_size: int = 10,
                  max_size: int = 16,
) -> pd.DataFrame:
    """
    Zwraca kopię df z nową kolumną `group_id` zawierającą numer grupy
    spełniającej warunki 10 ≤ n ≤ 16.
    """

    # 0. Normalizacja kodu do stałej długości 5 znaków (str)
    work = df.copy()
    work["_code_str"] = work[code_col].astype(str).str.zfill(5)

    accepted = {}                # {prefix: list[index]}
    small     = {}               # prefixy < min_size (do późniejszego łączenia)

    # 1. Rekurencyjne dzielenie dużych grup
    def split(prefix: str, idxs: list[int], depth: int):
        size = len(idxs)

        # a) gotowa grupa
        if min_size <= size <= max_size or depth == 5:
            accepted[prefix] = idxs
            return
        # b) za mała – zapisz do późniejszego łączenia
        if size < min_size:
            small[prefix] = idxs
            return
        # c) za duża – dziel głębiej
        next_depth = depth + 1
        sub_prefixes = work.loc[idxs, "_code_str"].str[:next_depth]
        for sub_pref, sub_idxs in work.loc[idxs].groupby(sub_prefixes).groups.items():
            split(sub_pref, list(sub_idxs), next_depth)

    # start od 2-cyfrowych
    for pref2, grp in work.groupby(work["_code_str"].str[:2]).groups.items():
        split(pref2, list(grp), depth=2)

    # 2. Łączenie małych grup w obrębie tych samych 2-cyfrowych prefiksów
    buckets = defaultdict(list)          # {pref2: [(pref, idxs), ...]}
    for p, idxs in small.items():
        buckets[p[:2]].append((p, idxs))

    for pref2, lst in buckets.items():
        # uporządkuj rosnąco wg wartości prefiksu → przybliżenie "sąsiedztwa"
        lst.sort(key=lambda x: int(x[0]))
        buf_idx, buf_pref = [], []

        for p, idxs in lst:
            buf_idx.extend(idxs)
            buf_pref.append(p)

            if len(buf_idx) >= min_size:
                # jeśli przypadkiem > max_size, tnij w kawałki
                while len(buf_idx) > max_size:
                    accepted[f"{pref2}_{len(accepted)}"] = buf_idx[:max_size]
                    buf_idx = buf_idx[max_size:]
                accepted["+".join(buf_pref)] = buf_idx
                buf_idx, buf_pref = [], []

        # resztka < 10 – dociągamy do ostatniej grupy z tego prefiksu
        if buf_idx:
            *_, last_key = [k for k in accepted.keys() if k.startswith(pref2)]
            accepted[last_key].extend(buf_idx)

    # 3. Ostateczne przypisanie numerów
    idx2gid = {}
    for gid, (_, lst) in enumerate(accepted.items()):
        for i in lst:
            idx2gid[i] = gid

    work["group_id"] = work.index.map(idx2gid)

    return work.drop(columns="_code_str")

In [43]:
obwody['kod'] = obwody['Kod pocztowy'].str.replace('-','').astype(int) 

obwody = assign_groups(obwody, code_col='kod')

Z artykułu:

"Większość grup spełniła założony docelowy rozmiar: 1 386 grup (62,8%) zawierało od 10 do 16
komisji, a 2 017 grup (91,3%) zawierało od 6 do 30 komisji. Większe grupy zazwyczaj odpowiadały 5
obszarom miejskim — na przykład takim jak Toruń czy Włocławek, gdzie pojedynczy kod pocztowy
obejmował całe miasto"

In [44]:
# między 10 a 16
sum((obwody['group_id'].value_counts()>=10) & (obwody['group_id'].value_counts()<=16))

1223

In [45]:
# między 6 a 30
sum((obwody['group_id'].value_counts()>=6) & (obwody['group_id'].value_counts()<=30))

1925

In [46]:
# duże grupy
obwody[obwody['group_id']==2150]

,Gmina,TERYT gminy,Numer,Kod pocztowy,kod,group_id
4100,m. Toruń,46301.0,1,87-100,87100,2150
4101,m. Toruń,46301.0,2,87-100,87100,2150
4102,m. Toruń,46301.0,3,87-100,87100,2150
4103,m. Toruń,46301.0,4,87-100,87100,2150
4104,m. Toruń,46301.0,5,87-100,87100,2150
...,...,...,...,...,...,...
4221,m. Toruń,46301.0,122,87-100,87100,2150
4222,m. Toruń,46301.0,123,87-100,87100,2150
4223,m. Toruń,46301.0,124,87-100,87100,2150
4224,m. Toruń,46301.0,125,87-100,87100,2150


In [47]:
# usun grupy mniej niz 3-komisjowe
obwody['liczba komisji w grupie']  = obwody.groupby('group_id').transform('count')['kod']
obwody = obwody[obwody['liczba komisji w grupie']>3]

In [48]:
len(obwody)

30553

# Wyniki głosowania

In [49]:
p = pd.read_csv('./protokoly_po_obwodach_utf8.csv', sep=';')
d = pd.read_csv('./protokoly_po_obwodach_w_drugiej_turze_utf8.csv', sep=';')


In [50]:
keep = ['Teryt Gminy', 'Nr komisji', 'Gmina', 'TRZASKOWSKI Rafał Kazimierz', 'NAWROCKI Karol Tadeusz']

In [51]:
p = p[keep].dropna()
d = d[keep].dropna()

In [52]:
p.head(2)

,Teryt Gminy,Nr komisji,Gmina,TRZASKOWSKI Rafał Kazimierz,NAWROCKI Karol Tadeusz
0,20101.0,1,m. Bolesławiec,361.0,287.0
1,20101.0,2,m. Bolesławiec,381.0,228.0


In [53]:
# polacz 1 i 2 tura

In [54]:
df = pd.merge(
    left=p, 
    right=d,
    how='left',
    left_on=['Teryt Gminy', 'Nr komisji'],
    right_on=['Teryt Gminy', 'Nr komisji'],
    suffixes = ('_1', '_2')
)

In [55]:
# polacz z obwodami

In [56]:
df = pd.merge(
    left=df, 
    right=obwody,
    how='left',
    left_on=['Teryt Gminy', 'Nr komisji'],
    right_on=['TERYT gminy', 'Numer'],
)


In [57]:
from copy import copy
df = copy(df.dropna())

# Reprodukcja Analizy z artykułu dla obu kandydatów

## 1. Nadmierne poparcie dla Karola Nawrockiego (względem mediany w ramach lokalnej grupy)

In [58]:
kandydat_A = "TRZASKOWSKI Rafał Kazimierz"
kandydat_B = "NAWROCKI Karol Tadeusz"

In [59]:
# mediana w grupie
df[kandydat_A + '_mediana_2'] = df.groupby('group_id')[kandydat_A+'_2'].transform('median')
df[kandydat_B + '_mediana_2'] = df.groupby('group_id')[kandydat_B+'_2'].transform('median')

In [60]:
# MAD w grupie
from scipy.stats import median_abs_deviation
def mad(series):
    """Median Absolute Deviation"""
    return median_abs_deviation(series)

df[kandydat_A + "_MAD_2"] = df.groupby("group_id")[kandydat_A+'_2'].transform(mad)
df[kandydat_B + "_MAD_2"] = df.groupby("group_id")[kandydat_B+'_2'].transform(mad)

In [61]:
# wspolczynnik k

In [62]:
df[kandydat_A +'_k_analiza_1'] = (df[kandydat_A +'_2'] - df[kandydat_A + '_mediana_2'])/df[kandydat_A + '_MAD_2']

In [63]:
df[kandydat_B +'_k_analiza_1'] = (df[kandydat_B +'_2'] - df[kandydat_B + '_mediana_2'])/df[kandydat_B + '_MAD_2']

In [64]:
k = 3

In [65]:
kandydat_A, sum(df[kandydat_A +'_k_analiza_1']>k)

('TRZASKOWSKI Rafał Kazimierz', 2794)

In [66]:
kandydat_B, sum(df[kandydat_B +'_k_analiza_1']>k)

('NAWROCKI Karol Tadeusz', 2015)

In [67]:
k = 2

In [68]:
kandydat_A, sum(df[kandydat_A +'_k_analiza_1']>k)

('TRZASKOWSKI Rafał Kazimierz', 4551)

In [69]:
kandydat_B, sum(df[kandydat_B +'_k_analiza_1']>k)

('NAWROCKI Karol Tadeusz', 3750)

### Wynik:
Dla k=2, takich grup, w których "za duże" poparcie ma Nawrocki jest 3750, a Trzaskowski 4551.

In [128]:
# anomalia na korzyść
df[kandydat_A + "_anomalia_1"] = df[kandydat_A +'_k_analiza_1']>2
df[kandydat_B + "_anomalia_1"] = df[kandydat_B +'_k_analiza_1']>2

## 2. Nadmierny względny wzrost poparcia dla Karola Nawrockiego między pierwszą a drugą turą, w porównaniu do odpowiedniego wzrostu poparcia dla Rafała Trzaskowskiego w tej samej grupie lokalnej;


In [129]:
# wzgledny wzrost między pierwszą a drugą turą
df[kandydat_A+ '_wzrost'] = df[kandydat_A+'_2']/df[kandydat_A+'_1']
df[kandydat_B+ '_wzrost'] = df[kandydat_B+'_2']/df[kandydat_B+'_1']

In [130]:
# roznica wzglednego roz_wzru miedzy kandydatami
df['roz_wzr_' + kandydat_A] =  df[kandydat_A + '_wzrost']  - df[kandydat_B + '_wzrost'] 
df['roz_wzr_' + kandydat_B] =  df[kandydat_B + '_wzrost']  - df[kandydat_A + '_wzrost'] 

In [131]:
# mediana i mad różnicy względnego wzrostu poparcia

df['roz_wzr_' + kandydat_A +'_mediana'] = df.groupby('group_id')['roz_wzr_' + kandydat_A].transform('median')
df['roz_wzr_' + kandydat_A + '_MAD'] =    df.groupby('group_id')['roz_wzr_' + kandydat_A].transform(mad)

In [132]:
df['roz_wzr_' + kandydat_B +'_mediana'] = df.groupby('group_id')['roz_wzr_' + kandydat_B].transform('median')
df['roz_wzr_' + kandydat_B + '_MAD'] =    df.groupby('group_id')['roz_wzr_' + kandydat_B].transform(mad)

In [133]:
df[kandydat_A +'_k_analiza_2'] = (df['roz_wzr_' + kandydat_A] - df['roz_wzr_' + kandydat_A +'_mediana'])/df['roz_wzr_' + kandydat_A + '_MAD']

In [134]:
df[kandydat_B +'_k_analiza_2'] = (df['roz_wzr_' + kandydat_B] - df['roz_wzr_' + kandydat_B +'_mediana'])/df['roz_wzr_' + kandydat_B + '_MAD']

In [135]:
k = 3

In [136]:
kandydat_A, sum(df[kandydat_A +'_k_analiza_2']>k)

('TRZASKOWSKI Rafał Kazimierz', 2106)

In [137]:
kandydat_B, sum(df[kandydat_B +'_k_analiza_2']>k)

('NAWROCKI Karol Tadeusz', 1669)

In [138]:
k = 2

In [139]:
kandydat_A, sum(df[kandydat_A +'_k_analiza_2']>k)

('TRZASKOWSKI Rafał Kazimierz', 3552)

In [140]:
kandydat_B, sum(df[kandydat_B +'_k_analiza_2']>k)

('NAWROCKI Karol Tadeusz', 3127)

"Nadmierny" względny wzrost poparcia dla Trzaskowskiego odnotowano w 3552 komisjach
"Nadmierny" względny wzrost poparcia dla Nawrockiego odnotowano w 3127 komisjach

In [141]:
# anomalia na korzyść kandydata
df[kandydat_A + "_anomalia_2"] = df[kandydat_A +'_k_analiza_2']>3
df[kandydat_B + "_anomalia_2"] = df[kandydat_B +'_k_analiza_2']>3

### 3 Komisje, w których Nawrocki uzyskał więcej głosów niż Trzaskowski w drugiej turze, mimo że mediana wyników w grupie wskazywała na przewagę Trzaskowskiego;

In [142]:
# wieksza mediana w grupie
df['wieksza_mediana_' + kandydat_A] = (df[kandydat_A + '_mediana_2'] >  df[kandydat_B + '_mediana_2']).astype(bool)
df['wieksza_mediana_' + kandydat_B] = (df[kandydat_B + '_mediana_2'] >  df[kandydat_A + '_mediana_2']).astype(bool)

In [143]:
df_wieksza_mediana_kandydat_A = df[df['wieksza_mediana_' + kandydat_A]]
df_wieksza_mediana_kandydat_B = df[df['wieksza_mediana_' + kandydat_B]]

In [144]:
kandydat_A, sum(df_wieksza_mediana_kandydat_A[kandydat_B+'_2']>df_wieksza_mediana_kandydat_A[kandydat_A+'_2'])

('TRZASKOWSKI Rafał Kazimierz', 1843)

In [145]:
kandydat_B, sum(df_wieksza_mediana_kandydat_B[kandydat_A+'_2']>df_wieksza_mediana_kandydat_B[kandydat_B+'_2'])

('NAWROCKI Karol Tadeusz', 2608)

W grupach, w których większą medianę miał Trzaskowski, 1843 razy wyższy wyniki uzyskał Nawrocki.

W grupach, w których większą medianę miał Nawrocki, 2608 razy wyższy wynik uzyskał Trzaskowski.

In [146]:
# anomalie na korzysc
df[kandydat_A + '_anomalia_3'] = df['wieksza_mediana_' + kandydat_B] & (df[kandydat_A + '_2'] > df[kandydat_B + '_2']) 
df[kandydat_B + '_anomalia_3'] = df['wieksza_mediana_' + kandydat_A] & (df[kandydat_B + '_2'] > df[kandydat_A + '_2']) 


## 4. Kandydat otrzymał mniej głosów w drugiej turze niż w pierwszej

In [147]:
kandydat_A, sum(df[kandydat_A + '_2']<df[kandydat_A + '_1'])

('TRZASKOWSKI Rafał Kazimierz', 128)

In [148]:
kandydat_B, sum(df[kandydat_B + '_2']<df[kandydat_B + '_1'])

('NAWROCKI Karol Tadeusz', 112)

W 128 komisjach Trzaskowski uzyskał mniej głosów w drugiej turze niż w pierwszej.

W 112 komisjach Nawrocki uzyskał mniej głosó w drugiej turze niż w pierwszej.

In [149]:
# Anomalie na korzysc
df[kandydat_A + '_anomalia_4'] = df[kandydat_B + '_2']<df[kandydat_B + '_1']
df[kandydat_B + '_anomalia_4'] = df[kandydat_A + '_2']<df[kandydat_A + '_1']

In [150]:
df[kandydat_A + '_razem_anomali_na_korzysc'] = df[[
    kandydat_A + '_anomalia_1', 
    kandydat_A + '_anomalia_2',
    kandydat_A + '_anomalia_3',
    kandydat_A + '_anomalia_4']].sum(axis=1)

In [151]:
df[kandydat_B + '_razem_anomali_na_korzysc'] = df[[
    kandydat_B + '_anomalia_1', 
    kandydat_B + '_anomalia_2',
    kandydat_B + '_anomalia_3',
    kandydat_B + '_anomalia_4']].sum(axis=1)

In [172]:
kandydat_A, sum(df[kandydat_A + '_razem_anomali_na_korzysc']>=0)

('TRZASKOWSKI Rafał Kazimierz', 30553)

In [173]:
kandydat_A, sum(df[kandydat_A + '_razem_anomali_na_korzysc']>=1)

('TRZASKOWSKI Rafał Kazimierz', 8161)

In [174]:
kandydat_A, sum(df[kandydat_A + '_razem_anomali_na_korzysc']>=2)

('TRZASKOWSKI Rafał Kazimierz', 1179)

In [175]:
kandydat_A, sum(df[kandydat_A + '_razem_anomali_na_korzysc']>=3)

('TRZASKOWSKI Rafał Kazimierz', 34)

In [176]:
df[df[kandydat_A + '_razem_anomali_na_korzysc']>3]

,Teryt Gminy,Nr komisji,Gmina_1,TRZASKOWSKI Rafał Kazimierz_1,NAWROCKI Karol Tadeusz_1,Gmina_2,TRZASKOWSKI Rafał Kazimierz_2,NAWROCKI Karol Tadeusz_2,Gmina,TERYT gminy,Numer,Kod pocztowy,kod,group_id,liczba komisji w grupie,TRZASKOWSKI Rafał Kazimierz_mediana_2,NAWROCKI Karol Tadeusz_mediana_2,TRZASKOWSKI Rafał Kazimierz_MAD_2,NAWROCKI Karol Tadeusz_MAD_2,TRZASKOWSKI Rafał Kazimierz_k_analiza_1,NAWROCKI Karol Tadeusz_k_analiza_1,TRZASKOWSKI Rafał Kazimierz_anomalia_1,NAWROCKI Karol Tadeusz_anomalia_1,TRZASKOWSKI Rafał Kazimierz_wzrost,NAWROCKI Karol Tadeusz_wzrost,roz_wzr_TRZASKOWSKI Rafał Kazimierz,roz_wzr_NAWROCKI Karol Tadeusz,roz_wzr_TRZASKOWSKI Rafał Kazimierz_mediana,roz_wzr_TRZASKOWSKI Rafał Kazimierz_MAD,roz_wzr_NAWROCKI Karol Tadeusz_mediana,roz_wzr_NAWROCKI Karol Tadeusz_MAD,TRZASKOWSKI Rafał Kazimierz_k_analiza_2,NAWROCKI Karol Tadeusz_k_analiza_2,TRZASKOWSKI Rafał Kazimierz_anomalia_2,NAWROCKI Karol Tadeusz_anomalia_2,wieksza_mediana_TRZASKOWSKI Rafał Kazimierz,wieksza_mediana_NAWROCKI Karol Tadeusz,TRZASKOWSKI Rafał Kazimierz_anomalia_3,NAWROCKI Karol Tadeusz_anomalia_3,TRZASKOWSKI Rafał Kazimierz_anomalia_4,NAWROCKI Karol Tadeusz_anomalia_4,TRZASKOWSKI Rafał Kazimierz_razem_anomali_na_korzysc,NAWROCKI Karol Tadeusz_razem_anomali_na_korzysc
5098,60903.0,4,gm. Bychawa,89.0,174.0,gm. Bychawa,260.0,163.0,gm. Bychawa,60903.0,4.0,23-100,23100.0,522.0,20.0,64.0,203.5,33.5,68.0,5.850746,-0.595588,True,False,2.921348,0.936782,1.984567,-1.984567,0.283991,0.291965,-0.283991,0.291965,5.824589,-5.824589,True,False,False,True,True,False,True,False,4,0
12616,140706.0,1,gm. Magnuszew,105.0,285.0,gm. Magnuszew,467.0,193.0,gm. Magnuszew,140706.0,1.0,26-910,26910.0,630.0,10.0,102.0,221.0,42.5,36.5,8.588235,-0.767123,True,False,4.447619,0.677193,3.770426,-3.770426,0.106758,0.364758,-0.106758,0.364758,10.044104,-10.044104,True,False,False,True,True,False,True,False,4,0
25866,261207.0,4,gm. Staszów,143.0,224.0,gm. Staszów,360.0,209.0,gm. Staszów,261207.0,4.0,28-200,28200.0,662.0,29.0,123.0,294.0,97.0,57.0,2.443299,-1.491228,True,False,2.517483,0.933036,1.584447,-1.584447,0.020685,0.200502,-0.020685,0.200502,7.799246,-7.799246,True,False,False,True,True,False,True,False,4,0


In [177]:
kandydat_B, sum(df[kandydat_B + '_razem_anomali_na_korzysc']>=1)

('NAWROCKI Karol Tadeusz', 6871)

In [178]:
kandydat_B, sum(df[kandydat_B + '_razem_anomali_na_korzysc']>=2)

('NAWROCKI Karol Tadeusz', 483)

In [179]:
kandydat_B, sum(df[kandydat_B + '_razem_anomali_na_korzysc']>=3)

('NAWROCKI Karol Tadeusz', 34)

In [180]:
kandydat_B, sum(df[kandydat_B + '_razem_anomali_na_korzysc']>=4)

('NAWROCKI Karol Tadeusz', 2)

In [182]:
df[df[kandydat_B + '_razem_anomali_na_korzysc']>3]

,Teryt Gminy,Nr komisji,Gmina_1,TRZASKOWSKI Rafał Kazimierz_1,NAWROCKI Karol Tadeusz_1,Gmina_2,TRZASKOWSKI Rafał Kazimierz_2,NAWROCKI Karol Tadeusz_2,Gmina,TERYT gminy,Numer,Kod pocztowy,kod,group_id,liczba komisji w grupie,TRZASKOWSKI Rafał Kazimierz_mediana_2,NAWROCKI Karol Tadeusz_mediana_2,TRZASKOWSKI Rafał Kazimierz_MAD_2,NAWROCKI Karol Tadeusz_MAD_2,TRZASKOWSKI Rafał Kazimierz_k_analiza_1,NAWROCKI Karol Tadeusz_k_analiza_1,TRZASKOWSKI Rafał Kazimierz_anomalia_1,NAWROCKI Karol Tadeusz_anomalia_1,TRZASKOWSKI Rafał Kazimierz_wzrost,NAWROCKI Karol Tadeusz_wzrost,roz_wzr_TRZASKOWSKI Rafał Kazimierz,roz_wzr_NAWROCKI Karol Tadeusz,roz_wzr_TRZASKOWSKI Rafał Kazimierz_mediana,roz_wzr_TRZASKOWSKI Rafał Kazimierz_MAD,roz_wzr_NAWROCKI Karol Tadeusz_mediana,roz_wzr_NAWROCKI Karol Tadeusz_MAD,TRZASKOWSKI Rafał Kazimierz_k_analiza_2,NAWROCKI Karol Tadeusz_k_analiza_2,TRZASKOWSKI Rafał Kazimierz_anomalia_2,NAWROCKI Karol Tadeusz_anomalia_2,wieksza_mediana_TRZASKOWSKI Rafał Kazimierz,wieksza_mediana_NAWROCKI Karol Tadeusz,TRZASKOWSKI Rafał Kazimierz_anomalia_3,NAWROCKI Karol Tadeusz_anomalia_3,TRZASKOWSKI Rafał Kazimierz_anomalia_4,NAWROCKI Karol Tadeusz_anomalia_4,TRZASKOWSKI Rafał Kazimierz_razem_anomali_na_korzysc,NAWROCKI Karol Tadeusz_razem_anomali_na_korzysc
11610,126101.0,95,m. Kraków,550.0,218.0,m. Kraków,540.0,1132.0,m. Kraków,126101.0,95.0,31-346,31346.0,691.0,14.0,814.5,424.5,174.0,46.0,-1.577586,15.380435,False,True,0.981818,5.192661,-4.210842,4.210842,-0.059256,0.158571,0.059256,0.158571,-26.181205,26.181205,False,True,True,False,False,True,False,True,0,4
17032,161105.0,9,gm. Strzelce Opolskie,311.0,107.0,gm. Strzelce Opolskie,223.0,416.0,gm. Strzelce Opolskie,161105.0,9.0,47-100,47100.0,1408.0,34.0,230.5,174.5,101.0,60.0,-0.074257,4.025000,False,True,0.717042,3.887850,-3.170809,3.170809,-0.656285,0.240919,0.656285,0.240919,-10.437204,10.437204,False,True,True,False,False,True,False,True,0,4
